In [1]:
library('dplyr')
library('effsize')
library('ggplot2')
library('foreign')
library('pscl')
library('ggrepel')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Classes and Methods for R developed in the
Political Science Computational Laboratory
Department of Political Science
Stanford University
Simon Jackman
hurdle and zeroinfl functions by Achim Zeileis



In [2]:
dfMDD = read.csv('../Data/dfMDDMDDCurrPrin_satisfaction_rdq_dx_demo.csv')
log = read.spss('../Data/DemosDx/Tx Day Consent Log_1.sav')
nrow(dfMDD)
demo = read.spss('../Data/DemosDx/Demographics Form_1.sav')
dx = read.spss('../Data/DemosDx/Diagnosis_1.sav')

Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav"):
“Undeclared level(s) 0, 11, 12 added in variable: DC_status_1”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav"):
“Duplicated levels in factor DC_status_1: Other”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav"):
“Undeclared level(s) 2 added in variable: SCID_1”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav"):
“Undeclared level(s) 0 added in variable: Referral_1”


[1] 844

Warning message in read.spss("../Data/DemosDx/Demographics Form_1.sav"):
“Undeclared level(s) 6 added in variable: Race_1”
Warning message in read.spss("../Data/DemosDx/Demographics Form_1.sav"):
“Undeclared level(s) 8 added in variable: Relationship_1”
Warning message in read.spss("../Data/DemosDx/Diagnosis_1.sav"):
“../Data/DemosDx/Diagnosis_1.sav: Long string value labels record found (record type 7, subtype 21), but ignored”
Warning message in read.spss("../Data/DemosDx/Diagnosis_1.sav"):
“Undeclared level(s) 22 added in variable: eatnos_1”
Warning message in read.spss("../Data/DemosDx/Diagnosis_1.sav"):
“Undeclared level(s) 10 added in variable: addhpr_1”


In [3]:
dfMDD$pre_cope_r = abs(dfMDD$pre_cope-2)
dfMDD$pre_pmh_r = abs(dfMDD$pre_pmh-2)
dfMDD$pre_fun_r = abs(dfMDD$pre_fun-2)
dfMDD$pre_well_r = abs(dfMDD$pre_well-2)
dfMDD$post_cope_r = abs(dfMDD$post_cope-2)
dfMDD$post_pmh_r = abs(dfMDD$post_pmh-2)
dfMDD$post_fun_r = abs(dfMDD$post_fun-2)
dfMDD$post_well_r = abs(dfMDD$post_well-2)

In [4]:
dfMDD$pre_dsym_sum = rowSums(dfMDD[grep('rdqpre_[1-9]_1|rdqpre_1[0-4]_1', colnames(dfMDD))])
dfMDD$pre_ndsym_sum = rowSums(dfMDD[grep('rdqpre_1[5-9]_1|rdqpre_2[0-5]_1', colnames(dfMDD))])
dfMDD$pre_cope_r_sum = rowSums(abs(dfMDD[grep('rdqpre_2[6-9]_1|rdqpre_30_1', colnames(dfMDD))]-2))
dfMDD$pre_pmh_r_sum = rowSums(abs(dfMDD[grep('rdqpre_3[1-9]_1|rdqpre_4[0-2]_1', colnames(dfMDD))]-2))
dfMDD$pre_fun_r_sum = rowSums(abs(dfMDD[grep('rdqpre_4[3-9]_1|rdqpre_5[0-2]_1', colnames(dfMDD))]-2))
dfMDD$pre_well_r_sum = rowSums(abs(dfMDD[grep('rdqpre_5[3-9]_1|rdqpre_69_1', colnames(dfMDD))]-2))

dfMDD$post_dsym_sum = rowSums(dfMDD[grep('rdqpost_[1-9]_1|rdqpost_1[0-4]_1', colnames(dfMDD))])
dfMDD$post_ndsym_sum = rowSums(dfMDD[grep('rdqpost_1[5-9]_1|rdqpost_2[0-5]_1', colnames(dfMDD))])
dfMDD$post_cope_r_sum = rowSums(abs(dfMDD[grep('rdqpost_2[6-9]_1|rdqpost_30_1', colnames(dfMDD))]-2))
dfMDD$post_pmh_r_sum = rowSums(abs(dfMDD[grep('rdqpost_3[1-9]_1|rdqpost_4[0-2]_1', colnames(dfMDD))]-2))
dfMDD$post_fun_r_sum = rowSums(abs(dfMDD[grep('rdqpost_4[3-9]_1|rdqpost_5[0-2]_1', colnames(dfMDD))]-2))
dfMDD$post_well_r_sum = rowSums(abs(dfMDD[grep('rdqpost_5[3-9]_1|rdqpost_69_1', colnames(dfMDD))]-2))
                                                            

In [13]:
(dfMDD$dep_resonder == 0 )

logical(0)

In [15]:
#Depression Responders
dfMDD$dep_responder = ifelse((dfMDD$pre_dsym - dfMDD$post_dsym)/(dfMDD$pre_dsym + 1e-10) >= 0.5, 1,0)
table(dfMDD$dep_responder)

dfMDD$pgi_responder_only  = ifelse(dfMDD$dep_responder == 0 & (dfMDD$imprv1 == 3 | dfMDD$imprv1 == 4), 1,0)
table(dfMDD$pgi_responder_only)

dfMDD$non_responder = ifelse(dfMDD$dep_responder == 0 & dfMDD$pgi_responder_only == 0, 1,0)
table(dfMDD$non_responder )


  0   1 
517 327 


  0   1 
574 270 


  0   1 
597 247 

In [16]:
#Admissino Descriptive 
admission_score = matrix(0,nrow = 4, ncol = 3)
facet = c('cope', 'pmh', 'fun', 'well')
status = c('dep_responder', 'pgi_responder_only', 'non_responder')

for (i in 1:4){
    for (j in 1:3){
        col = dfMDD[,grep(paste0('pre_', facet[i],'.*_sum'), colnames(dfMDD))]
        row = which(dfMDD[,grep(status[j], colnames(dfMDD))] == 1)
        mean = mean(col[row])
        sd = sd(col[row])
        admission_score[i,j] = paste0(round(mean,1), ' (', round(sd,1),')')

    }
}

admission_score

7.4 (2),7.3 (2.2),7.5 (2)
19.1 (4.2),18.6 (4.4),19.9 (3.6)
12.5 (3.9),13 (3.9),13.5 (3.6)
11.4 (2.8),11.4 (2.9),12 (2.2)


In [17]:
#Create a group variable
group = ifelse(dfMDD$dep_responder == 1, 'dep', 0)
group = ifelse(dfMDD$pgi_responder_only == 1, 'pgi', group)
group = ifelse(dfMDD$non_responder == 1, 'non', group)
table(group)

group
dep non pgi 
327 247 270 

In [18]:
#ANOVA admission
anova_admission = matrix(0, nrow = 4, ncol = 1)


for (i in 1:4){
    col = dfMDD[,grep(paste0('pre_', facet[i],'.*_sum'), colnames(dfMDD))]
    anova_results = summary(aov(col ~ group))
    df1 = anova_results[[1]]['group','Df']
    df2 = anova_results[[1]]['Residual','Df']
    F = anova_results[[1]]['group','F value']
    p = anova_results[[1]]['group','Pr(>F)']
    anova_admission[i] = paste0('F(',df1,',',df2,') = ', round(F,2), '; p = ', round(p,3))
    
}

anova_admission 


"F(2,841) = 0.44; p = 0.644"
"F(2,841) = 6.24; p = 0.002"
"F(2,841) = 4.69; p = 0.009"
"F(2,841) = 3.45; p = 0.032"


In [19]:
pairwise_t_test_result = function(col, x, y){
    t = t.test(col[group == x], col[group == y])
    if (t$p.value < 0.05/3){
        if (t$estimate[1] > t$estimate[2]){
            paste0(x,'>', y)
        }else{
            paste0(x,'<', y)
        }
    }else{
        paste0(x,'=', y)
    }
}


In [20]:
#T-test admission
t_test_table = matrix(0, nrow = 4, ncol = 1)

for (i in 1:4){
    col = dfMDD[,grep(paste0('pre_', facet[i],'.*_sum'), colnames(dfMDD))]
    t_test_table[i,1] = paste(
    pairwise_t_test_result(col, 'dep', 'pgi'),
    pairwise_t_test_result(col, 'dep', 'non'),
    pairwise_t_test_result(col, 'pgi', 'non'))
}

t_test_table

dep=pgi dep=non pgi=non
dep=pgi dep=non pgi<non
dep=pgi dep<non pgi=non
dep=pgi dep<non pgi=non


In [21]:
discharge_score = matrix(0,nrow = 4, ncol = 3)
facet = c('cope', 'pmh', 'fun', 'well')
status = c('dep_responder', 'pgi_responder_only', 'non_responder')

for (i in 1:4){
    for (j in 1:3){
        col = dfMDD[,grep(paste0('post_', facet[i],'.*_sum'), colnames(dfMDD))]
        row = which(dfMDD[,grep(status[j], colnames(dfMDD))] == 1)
        mean = mean(col[row])
        sd = sd(col[row])
        discharge_score[i,j] = paste0(round(mean,1), ' (', round(sd,1),')')

    }
}

discharge_score

3 (2),5 (2.1),6.6 (2.1)
7 (5),12.5 (5.1),17.4 (4.1)
4.9 (3.6),9.1 (4.1),11.7 (4.1)
4.5 (3.1),7.8 (3.4),10.6 (2.6)


In [22]:
#ANOVA discharge
anova_discharge = matrix(0, nrow = 4, ncol = 1)


for (i in 1:4){
    col = dfMDD[,grep(paste0('post_', facet[i],'.*_sum'), colnames(dfMDD))]
    anova_results = summary(aov(col ~ group))
    df1 = anova_results[[1]]['group','Df']
    df2 = anova_results[[1]]['Residual','Df']
    F = anova_results[[1]]['group','F value']
    p = anova_results[[1]]['group','Pr(>F)']
    anova_discharge[i] = paste0('F(',df1,',',df2,') = ', round(F,2), '; p = ', round(p,3))
    
}

anova_discharge


"F(2,841) = 219.57; p = 0"
"F(2,841) = 329.3; p = 0"
"F(2,841) = 223.17; p = 0"
"F(2,841) = 287.29; p = 0"


In [23]:
#T-test discharge
t_test_table_discharge = matrix(0, nrow = 4, ncol = 1)

for (i in 1:4){
    col = dfMDD[,grep(paste0('post_', facet[i],'.*_sum'), colnames(dfMDD))]
    t_test_table_discharge[i,1] = paste(
    pairwise_t_test_result(col, 'dep', 'pgi'),
    pairwise_t_test_result(col, 'dep', 'non'),
    pairwise_t_test_result(col, 'pgi', 'non'))
}

t_test_table_discharge 

dep<pgi dep<non pgi<non
dep<pgi dep<non pgi<non
dep<pgi dep<non pgi<non
dep<pgi dep<non pgi<non


In [24]:
i = 1
j = 1
pre = dfMDD[,grep(paste0('pre_', facet[i],'.*_sum'), colnames(dfMDD))]
post = dfMDD[,grep(paste0('post_', facet[i],'.*_sum'), colnames(dfMDD))]
row = which(dfMDD[,grep(status[j], colnames(dfMDD))] == 1)
t = t.test(pre[row], post[row])

t_value = round(t$statistic,2)
df = round(t$parameter, 2)
p_value = round(t$p.value, 3)

d = round(cohen.d(pre[row], post[row])$estimate, 2)

paste0('t(',df,') = ', t_value,'; p = ', p_value, '; d = ', d)



[1] "t(651.66) = 28.28; p = 0; d = 2.21"

In [25]:
#Change Score

change_score = matrix(0,nrow = 4, ncol = 3)
facet = c('cope', 'pmh', 'fun', 'well')
status = c('dep_responder', 'pgi_responder_only', 'non_responder')

for (i in 1:4){
    for (j in 1:3){
        pre = dfMDD[,grep(paste0('pre_', facet[i],'.*_sum'), colnames(dfMDD))]
        post = dfMDD[,grep(paste0('post_', facet[i],'.*_sum'), colnames(dfMDD))]
        row = which(dfMDD[,grep(status[j], colnames(dfMDD))] == 1)
        t = t.test(pre[row], post[row])
        t_value = round(t$statistic,2)
        df = round(t$parameter, 2)
        p_value = round(t$p.value, 3)
        if (p_value < 0.001)
            {p_value = ' < .001'}else{
            p_value = paste0(' = ', p_value)
        }

        d = round(cohen.d(pre[row], post[row])$estimate, 2)

        change_score[i,j] = paste0('t = ', t_value,'; p', p_value, '; d = ', d)


    }
}

change_score
# small (d = 0.2), medium (d = 0.5), and large (d = 0.8) 

t = 28.28; p < .001; d = 2.21,t = 12.58; p < .001; d = 1.08,t = 4.97; p < .001; d = 0.45
t = 33.5; p < .001; d = 2.62,t = 14.94; p < .001; d = 1.29,t = 7.26; p < .001; d = 0.65
t = 26.14; p < .001; d = 2.04,t = 11.45; p < .001; d = 0.99,t = 5.21; p < .001; d = 0.47
t = 30.09; p < .001; d = 2.35,t = 13.43; p < .001; d = 1.16,t = 6.09; p < .001; d = 0.55


In [39]:
#Responder Percentage

responder_percentage = matrix(0,nrow = 4, ncol = 3)
facet = c('cope', 'pmh', 'fun', 'well')
status = c('dep_responder', 'pgi_responder_only', 'non_responder')

for (i in 1:4){
    for (j in 1:3){
        pre = dfMDD[,grep(paste0('pre_', facet[i],'.*_sum'), colnames(dfMDD))]
        post = dfMDD[,grep(paste0('post_', facet[i],'.*_sum'), colnames(dfMDD))]
        row = which(dfMDD[,grep(status[j], colnames(dfMDD))] == 1)
        responder = sum((pre[row]-post[row])/pre[row] >= 0.5)/length(row)*100
        assign(paste0(facet[i],'_',status[j]), ifelse((pre[row]-post[row])/pre[row] >= 0.5, 1, 0))
        responder_percentage[i,j] =  paste0(round(responder,2),'%')

    }
}

responder_percentage

66.67%,28.89%,10.12%
70.03%,24.44%,3.24%
67.28%,26.67%,9.31%
67.89%,27.41%,5.26%


In [114]:
#Any Responder

any_responder_percentage = matrix(0,nrow = 1, ncol = 3)
for (j in 1:3){
    any_responder = 
    get(paste0('cope_',status[j])) + 
    get(paste0('pmh_',status[j])) + 
    get(paste0('fun_',status[j])) + 
    get(paste0('well_',status[j]))
    assign(paste0('any_responder_',status[j]), ifelse(any_responder > 0, 1, 0)) 
    any_responder_percentage[1,j] = paste0(round(sum(any_responder > 0)/length(any_responder)*100,2),'%')
}
any_responder_percentage

88.38%,52.59%,18.62%


In [75]:
responder_group = matrix(0,nrow = 4, ncol = 1)
for (i in 1:4){
    chi_data = data.frame(
        dep_responder = c(table(get(paste0(facet[i],'_dep_responder')))),
        pgi_responder_only = c(table(get(paste0(facet[i],'_pgi_responder_only')))),
        non_responder = c(table(get(paste0(facet[i],'_non_responder'))))
    )
    chisq_test_result = chisq.test(chi_data)
    
    v = round(chisq_test_result$statistic,1)
    p = round(chisq_test_result$p.value, 3)
    if (p < 0.001) p_text = 'p < .001' else p_text = paste0('p = ', p)
    responder_group[i,1] = paste0('X2 = ',v, '; ',p_text)
}
responder_group

X2 = 205; p < .001
X2 = 295.4; p < .001
X2 = 221.4; p < .001
X2 = 252.3; p < .001


In [121]:
#Any Respodner Chi-Square
chi_data = data.frame(
        dep_responder = c(table(any_responder_dep_responder)),
        pgi_responder_only = c(table(any_responder_pgi_responder_only)),
        non_responder = c(table(any_responder_non_responder))
    )
chisq_test_result = chisq.test(chi_data)
    
v = round(chisq_test_result$statistic,1)
p = round(chisq_test_result$p.value, 3)
if (p < 0.001) p_text = 'p < .001' else p_text = paste0('p = ', p)
paste0('X2 = ',v, '; ',p_text)

[1] "X2 = 281.1; p < .001"

In [110]:
pairwise_chi_test_result = function(col, x, y){
    chi_data = data.frame(
             group1 = c(table(get(paste0(col,'_',x)))),
             group2 = c(table(get(paste0(col,'_',y))))
        )
    chi = chisq.test(chi_data)
    
    if (chi$p.value < 0.05/3){
        if (chi$observed[2,1]/sum(chi$observed[,1]) > 
            chi$observed[2,2]/sum(chi$observed[,2])){
            paste0(x,'>', y)
        }else{
            paste0(x,'<', y)
        }
    }else{
        paste0(x,'=', y)
    }
}

In [117]:
#Pairwise Chi-Square
responder_group_pair = matrix(0,nrow = 4, ncol = 1)
for (i in 1:4){
    a = pairwise_chi_test_result(facet[i],'dep_responder','pgi_responder_only')
    b = pairwise_chi_test_result(facet[i],'dep_responder','non_responder')
    c = pairwise_chi_test_result(facet[i],'pgi_responder_only','non_responder')
    
    responder_group_pair[i,1] = paste(a,b,c)
}
responder_group_pair

dep_responder>pgi_responder_only dep_responder>non_responder pgi_responder_only>non_responder
dep_responder>pgi_responder_only dep_responder>non_responder pgi_responder_only>non_responder
dep_responder>pgi_responder_only dep_responder>non_responder pgi_responder_only>non_responder
dep_responder>pgi_responder_only dep_responder>non_responder pgi_responder_only>non_responder


In [120]:
#Any_respodner Pairwise Chi-Square
a = pairwise_chi_test_result('any_responder','dep_responder','pgi_responder_only')
b = pairwise_chi_test_result('any_responder','dep_responder','non_responder')
c = pairwise_chi_test_result('any_responder','pgi_responder_only','non_responder')
responder_group_pair = paste(a,b,c)
responder_group_pair

[1] "dep_responder>pgi_responder_only dep_responder>non_responder pgi_responder_only>non_responder"

In [155]:
grep('etohab_1', colnames(dfMDD))
grep('condct_1', colnames(dfMDD))
grep('mdd', colnames(dfMDD))
grep('nos', colnames(dfMDD))


[1] 139

[1] 258

[1] 175 176 177 178

[1] 162 168 180 181 194 210 228 234 245 252 256

In [183]:
dxnames = colnames(dfMDD)[grep('etohab_1', colnames(dfMDD)):grep('condct_1', colnames(dfMDD))]
dxnames_full = dxnames[-grep('nos|mdd', dxnames)]

dxnames_full


[1] "etohab_1"  "etohdp_1"  "amphab_1"  "amphdp_1"  "mjab_1"    "mjdep_1"  
  [7] "cokeab_1"  "cokedp_1"  "hallab_1"  "haldep_1"  "inhab_1"   "inhdep_1" 
 [13] "nic_1"     "opioab_1"  "opiodp_1"  "pcpab_1"   "pcpdep_1"  "sedab_1"  
 [19] "seddep_1"  "othrab_1"  "othrdp_1"  "polyab_1"  "polydp_1"  "z_1"      
 [25] "zform_1"   "zaff_1"    "delus_1"   "brief_1"   "adjdep_1"  "adjanx_1" 
 [31] "adjmix_1"  "adjcnd_1"  "adjcmo_1"  "adjun_1"   "dysth_1"   "gmcful_1" 
 [37] "gmcprt_1"  "bp1hyp_1"  "bp1man_1"  "bp1mix_1"  "bp1dpp_1"  "bp1dnp_1" 
 [43] "bp2hyp_1"  "bp2dep_1"  "bp2mix_1"  "bpmod_1"   "cyclo_1"   "panic_1"  
 [49] "pda_1"     "awopd_1"   "spec4_1"   "specm_1"   "socal4_1"  "socalm_1" 
 [55] "ocd4_1"    "ocdm_1"    "ptsd4_1"   "ptsdm_1"   "acute_1"   "gad4_1"   
 [61] "gadm_1"    "gmcanx_1"  "subpan_1"  "subspc_1"  "subocd_1"  "subsoc_1" 
 [67] "subpts_1"  "subgad_1"  "anxmix_1"  "anxoth_1"  "ied_1"     "gamble_1" 
 [73] "trich_1"   "cbuy_1"    "csex_1"    "cexer_1"   "comput_1"  "klepto_1" 
 [79] "pyro_1"    "anor_1"    "bulim_1"   "binge_1"   "subbul_1"  "subanr_1" 
 [85] "para_1"    "zoid_1"    "typal_1"   "anti_1"    "bord_1"    "hist_1"   
 [91] "narc_1"    "avoid_1"   "depend_1"  "obcomp_1"  "somat_1"   "undiff_1" 
 [97] "conver_1"  "pain_1"    "hypoch_1"  "bdd_1"     "addcom_1"  "addatt_1" 
[103] "addhpr_1"  "adltadd_1" "condct_1"

In [202]:
dfMDD[,'socal4_1']

[1] Curr        PR          Curr        Curr        Curr        Not present
  [7] Not present Curr        Not present Not present Not present Not present
 [13] Not present Not present Not present Not present Not present Not present
 [19] Not present Curr        Not present Not present Curr        Not present
 [25] Not present Not present Not present Not present Not present Not present
 [31] Not present Curr        Not present Curr        Not present Not present
 [37] Not present Not present Not present Not present Not present Not present
 [43] Curr        Not present Not present Not present Curr        Not present
 [49] Curr        Not present Curr        Not present Not present Curr       
 [55] Not present Not present Not present Curr        Not present Not present
 [61] Not present Not present Not present Not present Not present Not present
 [67] Not present Not present Not present Not present Not present Not present
 [73] Curr        Curr        Curr        Not present Curr        Not present
 [79] Curr        Not present Curr        Not present Not present Curr       
 [85] Not present Not present Not present Curr        Curr        Not present
 [91] Not present Curr        Not present Not present Not present Curr       
 [97] Curr        Not present Not present Curr        Curr        Not present
[103] Not present Not present Curr        Not present Curr        Not present
[109] Not present Not present Not present Curr        Not present Not present
[115] Not present Not present Not present Not present Not present Curr       
[121] Not present Not present Not present Not present Not present Not present
[127] Curr        Not present Not present Curr        Not present Not present
[133] Curr        Not present PR          Curr        Not present Curr       
[139] Not present Not present Not present Not present Not present Curr       
[145] Curr        Not present Not present Not present Not present Not present
[151] Not present Curr        Curr        Not present Not present Not present
[157] Not present Not present Curr        Not present Not present Curr       
[163] Past        Not present Not present Not present Not present Not present
[169] Not present Curr        Not present Curr        Not present Not present
[175] Not present Not present Not present Not present Curr        Not present
[181] Not present Not present Curr        Not present Curr        Not present
[187] Not present Not present Not present Not present Curr        Not present
[193] Not present Not present Not present Curr        Curr        Not present
[199] Not present Not present Curr        Curr        Not present Not present
[205] Not present Not present Not present Not present Not present Not present
[211] Not present Not present Not present Not present Curr        Curr       
[217] Curr        Not present Not present Curr        Not present Curr       
[223] Not present Curr        Not present Not present Curr        Not present
[229] Curr        Not present Not present Not present Not present Not present
[235] Not present Curr        Curr        Not present Curr        Curr       
[241] Curr        Not present Not present Not present Not present Not present
[247] Not present Not present Not present Not present Curr        Not present
[253] Not present Curr        Not present Not present Past        Not present
[259] Not present Curr        Curr        Curr        Curr        Not present
[265] Not present Not present Not present Not present Not present Not present
[271] Curr        Not present Curr        Not present Curr        Not present
[277] Not present Curr        Not present Not present Curr        Curr       
[283] Not present Not present Not present Not present Not present Not present
[289] Not present Not present Not present Not present Not present Not present
[295] Not present Not present Not present Not present Not present Not present
[301] Curr        Not present Not present Curr        Not present Not present
[307] Not present Not pr

In [205]:
dfMDD_pure = 
dfMDD %>%
filter_at(vars(dxnames_full[1:length(dxnames_full)]), 
          all_vars(. == 'Not present' | .== 'Past' | .== 'PR'))
dfMDD_pure


X,ID1,rdqpre_1_1,rdqpre_2_1,rdqpre_3_1,rdqpre_4_1,rdqpre_5_1,rdqpre_6_1,rdqpre_7_1,rdqpre_8_1,⋯,pre_well_r_sum,post_dsym_sum,post_ndsym_sum,post_cope_r_sum,post_pmh_r_sum,post_fun_r_sum,post_well_r_sum,dep_responder,pgi_responder_only,non_responder
<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1384,2,1,2,2,0,1,2,2,⋯,11,6,9,2,1,11,6,1,0,0
16,1423,2,2,2,2,0,1,2,2,⋯,9,8,4,5,13,9,5,1,0,0
41,1581,2,2,2,0,0,2,0,2,⋯,13,10,6,5,0,5,5,0,0,1
124,1932,2,2,2,2,0,2,2,2,⋯,12,11,6,4,10,10,8,0,1,0
144,1998,2,2,2,2,0,1,2,1,⋯,14,13,12,5,12,8,8,0,1,0
149,2053,2,2,2,0,2,2,1,2,⋯,14,16,3,5,21,11,14,0,1,0
158,2080,2,2,2,0,2,2,2,2,⋯,12,20,14,10,21,17,14,0,0,1
162,2101,2,2,2,2,2,2,2,2,⋯,13,13,8,5,9,9,3,0,1,0
165,2106,2,1,1,2,0,1,0,1,⋯,2,1,1,1,2,2,1,1,0,0


[1]  0.66666667  0.61904762  0.50000000  0.42105263  0.40909091  0.23809524
 [7]  0.09090909  0.45833333  0.91666667  0.21052632  0.26086957  0.68421053
[13]  0.00000000  0.30000000  0.84615385  0.42105263  0.23529412  0.22222222
[19]  0.00000000  0.77777778  0.75000000  0.38095238  0.37500000  0.52380952
[25]  0.43750000  0.47826087  0.47058824  0.91304348  0.77272727  0.89473684
[31]  0.40000000  0.70588235  0.42857143  0.29166667  0.16666667  0.16666667
[37]  0.18181818  0.35000000  0.70833333  0.84210526  0.14285714  0.31578947
[43]  0.16666667  1.00000000  0.73333333  0.33333333  1.00000000  0.87500000
[49]  0.47368421  0.20000000  0.25000000  0.84210526 -0.23529412  0.84615385
[55]  0.63636364  0.26315789  1.00000000  0.85714286  1.00000000  0.00000000
[61]  0.45454545  0.28571429  0.21428571  0.66666667  0.04166667  0.65000000
[67]  0.70000000  0.40000000  0.33333333  0.76923077  0.31578947  0.90909091
[73]  0.15000000  0.27272727  0.47619048  0.76923077  0.41176471  0.61904762
[79]  0.17391304  0.80000000  0.34782609  0.35294118  0.57894737  0.61904762
[85]  0.68750000  0.66666667  0.87500000  0.44444444  1.00000000  0.81250000
[91]  0.81818182  0.63636364

In [243]:

resp = ifelse((dfMDD_pure$pre_dsym - dfMDD_pure$post_dsym)/(dfMDD_pure$pre_dsym + 1e-10) >= 0.5, 1, 0)
sum((dfMDD_pure$pre_dsym - dfMDD_pure$post_dsym)/(dfMDD_pure$pre_dsym + 1e-10) >= 0.5)/length(dfMDD_pure$pre_dsym)
sum(dfMDD_pure$imprv1 >= 3)/nrow(dfMDD_pure)
sum(dfMDD_pure$imprv1[resp == 0] >= 3)/nrow(dfMDD_pure)
sum(dfMDD_pure$imprv1[resp == 0] >= 3)/nrow(dfMDD_pure[resp == 0,])

[1] 0.4456522

[1] 0.7608696

[1] 0.3586957

[1] 0.6470588

In [240]:
nrow(dfMDD_pure[dfMDD])

[1] 92

In [245]:
nrow(dfMDD_pure[resp == 0,])

[1] 51

In [247]:

resp = ifelse((dfMDD$pre_dsym - dfMDD$post_dsym)/(dfMDD$pre_dsym + 1e-10) >= 0.5, 1, 0)
sum((dfMDD$pre_dsym - dfMDD$post_dsym)/(dfMDD$pre_dsym + 1e-10) >= 0.5)/length(dfMDD$pre_dsym)
sum(dfMDD$imprv1 >= 3)/nrow(dfMDD)
sum(dfMDD$imprv1[resp == 0] >= 3)/nrow(dfMDD[resp == 0,])

[1] 0.3874408

[1] 0.6741706

[1] 0.5222437

In [250]:
nrow(dfMDD[resp == 0,])

[1] 517

In [229]:
sum(dfMDD$imprv1[resp == 0] >= 3)/nrow(dfMDD[resp == 0,])

[1] 0.5222437